First, import needed modules

In [ ]:
import numpy as np
import pandas as pd
import os
import geopandas as gpd
import matplotlib.pyplot as plt
import shapely.speedups
from shapely.ops import unary_union

Initalize variables

In [ ]:
shapely.speedups.enable()
path = os.getcwd()
print(path)

Load shapefile with municipality and state boundaries and plot it for inspection

In [ ]:
municipalities = gpd.read_file(f'{path}/data/geodata/VG250_GEM.shp')
municipalities.index.rename('ID', inplace=True)
states = gpd.read_file(f'{path}/data/geodata/VG250_LAN.shp')
municipalities.plot()

Inspect the dataframe: GEN has the municipalities' names, geometry are the polygons.


In [ ]:
municipalities.head()

Check for duplicates in terms of the AGS (Amtlicher Gemeindeschlüssel) -> there are duplicates.

In [ ]:
municipalities['ags_dup'] = municipalities.duplicated(subset=['AGS'], keep=False)
municipalities.head()

Merge shapes for the duplicate municipalities

In [ ]:
duplicates = municipalities[municipalities['ags_dup'] == True] # create duplicates df
duplicates.insert(loc=len(duplicates.columns), column='union', value=0) # insert a new 'union column
duplicates.sort_values(by=['GEN'], inplace=True) # sort by city name
for i in range(0, len(duplicates), 2): # iterate over all duplicate cities
    city_name = duplicates.iloc[i, 6] # get the city name
    duplicate = duplicates[duplicates['GEN'] == city_name] # get geometries of both polygons for a given city
    duplicates['union'].loc[duplicates.index[i]]  = duplicate.unary_union # combine the shapes
duplicates = duplicates[duplicates['union'] != 0 ] # drop the duplicates, keep only the combined shapes
duplicates = duplicates.drop(columns=['geometry']) # drop the geometry column
duplicates.rename(columns = {'union':'geometry'}, inplace = True)   #rename the union column to geometry
duplicates.head()

Concatenate the duplicates df with the combined shapes to the municipalities df

In [ ]:
municipalities = municipalities[municipalities['ags_dup'] == False]
municipalities = gpd.GeoDataFrame(pd.concat([municipalities, duplicates], ignore_index=True), crs=[municipalities, duplicates][0].crs)
municipalities.sort_values(by=['GEN'], inplace=True) # sort by city name
municipalities = municipalities.drop(columns=['ags_dup'])
municipalities.to_csv(f'{path}/data/treatment.csv', encoding = 'utf-8-sig')
municipalities.head()

The data use the European Terrestrial Reference System 1989 as coordinate reference system (CRS), units are metres

In [ ]:
municipalities.crs

Load shapefile with power lines and plot it for inspection, as the CRS of powerlines is EPSG 4326 (Degrees), convert to EPSG 25832 (Meters)

In [ ]:
powerlines = gpd.read_file(f'{path}/data/geodata/powerlines.shp')
powerlines = powerlines.to_crs(epsg=25832)
powerlines.head()

Check CRS conversion

In [ ]:
powerlines.crs

Plot for inspection

In [ ]:
powerlines.plot()

Combine municipalities and powerlines in a plot

In [ ]:
powerlines.set_geometry('geometry')
fig, ax = plt.subplots(figsize=(10, 8), dpi=300)
ax.set_aspect('equal')
ax.set_axis_off()
municipalities.plot(ax=ax, color='lightblue', edgecolor='blue', lw=0.01, zorder=1)
states.boundary.plot(ax=ax, color='black', lw = 0.1, zorder=2)
powerlines.plot(ax=ax, color='red', lw=2, zorder=3)
plt.suptitle('BBPlG 2013 projects', fontsize=20)
plt.savefig(f'{path}/figures/BBPLG2013_projects.png')
plt.close()

Now construct the treatment indicator by checking if any of the power lines *directly* intersect with municipality polygons

In [ ]:
municipalities.insert(loc=len(municipalities.columns), column='treated_0', value=0)
for i in range(len(municipalities)):
    municipalities['treated_0'].loc[municipalities.index[i]] = any(powerlines['geometry'].intersects(municipalities['geometry'].values[i]))

Plot the municipalities that directly intersect with a powerline

In [ ]:
powerlines.set_geometry('geometry')
municipalities_intersected = municipalities[municipalities['treated_0'] == True] #drops all untreated munipalities
fig, ax = plt.subplots(figsize=(10, 8), dpi=300)
ax.set_aspect('equal')
ax.set_axis_off()
municipalities.plot(ax=ax, color='lightblue', edgecolor='blue', lw=0.01, zorder=1)
states.boundary.plot(ax=ax, color='darkblue', lw = 0.1, zorder=2)
municipalities_intersected.plot(ax=ax, color='orange', edgecolor='darkorange', lw=0.01, zorder=3)
powerlines.plot(ax=ax, color='red', lw=2, zorder=4)
plt.suptitle('Municipalities affected by BBPlG 2013 projects', fontsize=20)
plt.title('direct line', fontsize=14)
plt.savefig(f'{path}/figures/BBPLG2013_treated_0.png')
plt.close()


Create buffers around the straight lines to create treatment indicator for different corridor widths

In [ ]:
for buffer in (5, 10, 15, 25, 50):
    try:
        powerlines.insert(loc=len(powerlines.columns), column=f'geometry_{buffer*2}', value=0) #buffer is added on both sides -> x2 to simplify
    except Exception:
        pass
    powerlines[f'geometry_{buffer*2}'] = powerlines['geometry'].buffer(buffer*1000)
powerlines.head()

Now generate treatment indicators for all buffer sizes

In [ ]:
for buffer in (5, 10, 15, 25, 50):
    try:
        municipalities.insert(loc=len(municipalities.columns), column=f'treated_{buffer*2}', value=0)
    except Exception:
        pass
    for i in range(len(municipalities)):
        municipalities[f'treated_{buffer*2}'].loc[municipalities.index[i]] = any(powerlines[f'geometry_{buffer*2}'].intersects(municipalities['geometry'].values[i]))
municipalities.head()

Plot for inspection

In [ ]:
#powerlines.set_geometry('geometry_50')
municipalities_intersected50 = municipalities[municipalities['treated_50'] == True] #drops all untreated munipalities
fig, ax = plt.subplots(figsize=(10, 8), dpi=300)
ax.set_aspect('equal')
ax.set_axis_off()
municipalities.plot(ax=ax, color='lightblue', zorder=1)
states.boundary.plot(ax=ax, color='darkblue', lw = 0.1, zorder=2)
municipalities_intersected50.plot(ax=ax, color='orange', edgecolor='darkorange', lw=0.01, zorder=3)
powerlines.plot(ax=ax, color='red', lw=2, zorder=4)
plt.suptitle('Municipalities affected by BBPlG 2013 projects', fontsize=20)
plt.title('with a 50km buffer', fontsize=14)
plt.savefig(f'{path}/figures/BBPLG2013_treated_50.png')
plt.close()

Export treatment dataset from municipalities dataframe

In [ ]:
treatment = pd.DataFrame(municipalities)
treatment = treatment.reindex(columns=['AGS', 'GEN', 'treated_0', 'treated_10', 'treated_20', 'treated_30', 'treated_50', 'treated_100'])
treatment = treatment.set_index('AGS')
treatment = treatment.replace({True:1, False:0})
treatment.to_csv(f'{path}/data/treatment.csv', encoding = 'utf-8-sig')
treatment.head()